**Цель работы:**

Для рекомендательной системы мы хотим предсказывать вероятность того, что пользователь после первого прослушивания песни будет слушать её снова. В данном случае, целевая переменная 'target' указывает на наличие повторных прослушиваний: значение 1 означает, что повторные прослушивания есть, а значение 0 означает, что их нет.

Таким образом, задача состоит в том, чтобы предсказывать вероятность того, что для конкретной пары пользователя и песни (обозначенной, например, столбцами 'msno' и 'song_id') будет повторное прослушивание ('target' = 1).

# ПОДГОТОВКА ДАННЫХ

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!7z x /content/drive/MyDrive/train.csv.7z
!7z x /content/drive/MyDrive/songs.csv.7z
!7z x /content/drive/MyDrive/song_extra_info.csv.7z
!7z x /content/drive/MyDrive/members.csv.7z
!7z x /content/drive/MyDrive/sample_submission.csv.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/                                 1 file, 106420688 bytes (102 MiB)

Extracting archive: /content/drive/MyDrive/train.csv.7z
--
Path = /content/drive/MyDrive/train.csv.7z
Type = 7z
Physical Size = 106420688
Headers Size = 122
Method = LZMA2:24
Solid = -
Blocks = 1

  0%    
Would you like to replace the existing file:
  Path:     ./train.csv
  Size:     971675848 bytes (927 MiB)
  Modified: 2017-09-22 00:08:34
with the file from archive:
  Path:     train.csv
  Size:     971675848 bytes (927 MiB)
  Modified: 2017-09-22 00:08:34
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? 
7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 20

In [3]:
import pandas as pd
train_data = pd.read_csv("train.csv")

Для начала проверим все три файла на наличие пропусков и заполним их, в соответствии с типом данных.

In [4]:
train_data.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7377418 entries, 0 to 7377417
Data columns (total 6 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   msno                object
 1   song_id             object
 2   source_system_tab   object
 3   source_screen_name  object
 4   source_type         object
 5   target              int64 
dtypes: int64(1), object(5)
memory usage: 337.7+ MB


In [6]:
train_data.isnull().any()

msno                  False
song_id               False
source_system_tab      True
source_screen_name     True
source_type            True
target                False
dtype: bool

In [7]:
train_data[['source_type', 'source_screen_name', 'source_system_tab']] = train_data[['source_type', 'source_screen_name', 'source_system_tab']].fillna('UnKnown')

In [8]:
train_data.isnull().any()

msno                  False
song_id               False
source_system_tab     False
source_screen_name    False
source_type           False
target                False
dtype: bool

In [9]:
import numpy as np
train_data = np.array(train_data)
train_data= np.squeeze(train_data)
train_data.shape

(7377418, 6)

In [10]:
members_data = pd.read_csv("members.csv")
members_data.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


In [ ]:
members_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34403 entries, 0 to 34402
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   msno                    34403 non-null  object
 1   city                    34403 non-null  int64 
 2   bd                      34403 non-null  int64 
 3   gender                  14501 non-null  object
 4   registered_via          34403 non-null  int64 
 5   registration_init_time  34403 non-null  int64 
 6   expiration_date         34403 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 1.8+ MB


In [ ]:
members_data.isnull().any()

msno                      False
city                      False
bd                        False
gender                     True
registered_via            False
registration_init_time    False
expiration_date           False
dtype: bool

In [11]:
members_data[['gender']] = members_data[['gender']].fillna(0)

In [ ]:
members_data.isnull().any()

msno                      False
city                      False
bd                        False
gender                    False
registered_via            False
registration_init_time    False
expiration_date           False
dtype: bool

In [ ]:
members_data.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,0,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,0,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,0,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,0,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,0,4,20170126,20170613


In [12]:
import numpy as np
members_data = np.array(members_data)
members_data= np.squeeze(members_data)
members_data.shape

(34403, 7)

In [13]:
songs_data = pd.read_csv("songs.csv")
songs_data.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [ ]:
songs_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2296320 entries, 0 to 2296319
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   song_id      object 
 1   song_length  int64  
 2   genre_ids    object 
 3   artist_name  object 
 4   composer     object 
 5   lyricist     object 
 6   language     float64
dtypes: float64(1), int64(1), object(5)
memory usage: 122.6+ MB


In [ ]:
songs_data.isnull().any()

song_id        False
song_length    False
genre_ids       True
artist_name    False
composer        True
lyricist        True
language        True
dtype: bool

In [14]:
songs_data[['genre_ids','composer','lyricist']] = songs_data[['genre_ids','composer','lyricist']].fillna('UnKnown')
songs_data[['language','genre_ids']] = songs_data[['language','genre_ids']].fillna(0)

In [ ]:
songs_data.isnull().any()

song_id        False
song_length    False
genre_ids      False
artist_name    False
composer       False
lyricist       False
language       False
dtype: bool

In [15]:
import numpy as np
songs_data = np.array(songs_data)
songs_data= np.squeeze(songs_data)
songs_data.shape

(2296320, 7)

Объединим все три файла в один датафрейм.

In [16]:
songs_df = pd.DataFrame(songs_data, columns=['song_id', 'song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist', 'language'])
members_df = pd.DataFrame(members_data, columns=['msno', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date'])
train_df = pd.DataFrame(train_data, columns=['msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 'target'])

In [17]:
merged_df = pd.merge(train_df, songs_df, on='song_id', how='left')
merged_df = pd.merge(merged_df, members_df, on='msno', how='left')

In [ ]:
merged_df.shape

(7377418, 18)

In [ ]:
merged_df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,registration_init_time,expiration_date
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,206471,359,Bastille,Dan Smith| Mark Crew,UnKnown,52.0,1,0,0,7,20120102,20171005
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,284584,1259,Various Artists,UnKnown,UnKnown,52.0,13,24,female,9,20110525,20170911
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,225396,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,UnKnown,52.0,13,24,female,9,20110525,20170911
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,255512,1019,Soundway,Kwadwo Donkoh,UnKnown,-1.0,13,24,female,9,20110525,20170911
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,187802,1011,Brett Young,Brett Young| Kelly Archer| Justin Ebach,UnKnown,52.0,1,0,0,7,20120102,20171005


In [18]:
import pickle

with open("/content/drive/MyDrive/merged_df.pkl", 'wb') as f:
    pickle.dump(merged_df, f)

print("DataFrame успешно сохранен в файл", 'merged_df.pkl')

DataFrame успешно сохранен в файл merged_df.pkl


In [ ]:
print(merged_df.columns)

Index(['msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'target', 'song_length', 'genre_ids', 'artist_name',
       'composer', 'lyricist', 'language', 'city', 'bd', 'gender',
       'registered_via', 'registration_init_time', 'expiration_date'],
      dtype='object')


In [ ]:
print(merged_df['target'].value_counts())

target
1    3714656
0    3662762
Name: count, dtype: int64


# ОБУЧЕНИЕ

In [1]:
import pickle

with open("/content/drive/MyDrive/merged_df.pkl", 'rb') as f:
    merged_df = pickle.load(f)

print("DataFrame успешно загружен из файла", "merged_df.pkl")


DataFrame успешно загружен из файла merged_df.pkl


**Feature engeneering**

Заполняем пропуски, если такие образовались при конкатенации датафреймов и переводим цифры, представленные строковвыми данными, в инт.

In [2]:
mean_song_length = merged_df['song_length'].mean()
merged_df['song_length'] = merged_df['song_length'].fillna(mean_song_length)
merged_df['song_length'] = merged_df['song_length'].astype(int)
mean_song_length = merged_df['language'].mean()
merged_df['language'] = merged_df['language'].fillna(mean_song_length)
merged_df['language'] = merged_df['language'].astype(int)
merged_df['target'] = merged_df['target'].astype(int)
merged_df['city'] = merged_df['city'].astype(int)
merged_df['bd'] = merged_df['bd'].astype(int)
merged_df['registered_via'] = merged_df['registered_via'].astype(int)
merged_df['registration_init_time'] = merged_df['registration_init_time'].astype(int)
merged_df['expiration_date'] = merged_df['expiration_date'].astype(int)

Удаляем столбцы с длинными, слишком вариативными названиями и текстовыми данными, так как их будет проблематично кодировать.

In [3]:
merged_df=merged_df.drop(columns=['msno', 'song_id','artist_name','composer','lyricist'])

In [4]:
import pandas as pd

def convert_first_id(x):
    x = str(x)
    try:
        if '|' in x:
            return int(x.split('|')[0])
        return int(x)
    except ValueError:
        return 0

merged_df['genre_ids'] = merged_df['genre_ids'].apply(convert_first_id)


Кодируем one-hot-encoding оставшиеся категориальные переменные.

In [6]:
import pandas as pd
merged_df = pd.get_dummies(merged_df, columns=['source_system_tab', 'source_screen_name', 'source_type', 'gender'])

In [7]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7377418 entries, 0 to 7377417
Data columns (total 55 columns):
 #   Column                                   Dtype
---  ------                                   -----
 0   target                                   int64
 1   song_length                              int64
 2   genre_ids                                int64
 3   language                                 int64
 4   city                                     int64
 5   bd                                       int64
 6   registered_via                           int64
 7   registration_init_time                   int64
 8   expiration_date                          int64
 9   source_system_tab_UnKnown                bool 
 10  source_system_tab_discover               bool 
 11  source_system_tab_explore                bool 
 12  source_system_tab_listen with            bool 
 13  source_system_tab_my library             bool 
 14  source_system_tab_notification           bool 
 15

Подготовленный датасет для обучения выглядит следующим способом:

In [8]:
merged_df.head()

,target,song_length,genre_ids,language,city,bd,registered_via,registration_init_time,expiration_date,source_system_tab_UnKnown,...,source_type_my-daily-playlist,source_type_online-playlist,source_type_radio,source_type_song,source_type_song-based-playlist,source_type_top-hits-for-artist,source_type_topic-article-playlist,gender_0,gender_female,gender_male
0,1,206471,359,52,1,0,7,20120102,20171005,False,...,False,True,False,False,False,False,False,True,False,False
1,1,284584,1259,52,13,24,9,20110525,20170911,False,...,False,False,False,False,False,False,False,False,True,False
2,1,225396,1259,52,13,24,9,20110525,20170911,False,...,False,False,False,False,False,False,False,False,True,False
3,1,255512,1019,-1,13,24,9,20110525,20170911,False,...,False,False,False,False,False,False,False,False,True,False
4,1,187802,1011,52,1,0,7,20120102,20171005,False,...,False,True,False,False,False,False,False,True,False,False


Разбиваем данные на обучающую и тестовую выборки.

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(merged_df, test_size=0.2, random_state=42)

print("Размер обучающего набора:", X_train.shape)
print("Размер тестового набора:", X_test.shape)

Размер обучающего набора: (5901934, 55)
Размер тестового набора: (1475484, 55)


Отделяем столбец таргета для обучения.

In [10]:
y_train = X_train['target']
X_train = X_train.drop('target', axis=1)
y_test = X_test['target']
X_test = X_test.drop('target', axis=1)

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [32]:
X_train.shape

(5901934, 54)

In [12]:
import pickle

with open("/content/drive/MyDrive/X_train.pkl", 'wb') as f:
    pickle.dump(X_train, f)
with open("/content/drive/MyDrive/X_test.pkl", 'wb') as f:
    pickle.dump(X_test, f)

**Решающие деревья**

Обучаем алгоритм решающих деревьев.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

DecisionTree = DecisionTreeClassifier(max_depth=None,min_samples_leaf=1,min_samples_split=5)
DecisionTree.fit(X_train,y_train)
train_pred = DecisionTree.predict(X_train)
test_pred = DecisionTree.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
DecisionTree_train_acc = accuracy_score(y_train,train_pred)
DecisionTree_test_acc = accuracy_score(y_test,test_pred)
print("Training Accuracy : ",DecisionTree_train_acc)
print("Test Accuracy : ",DecisionTree_test_acc)
print("Classification Report:\n",classification_report(y_train,train_pred))

Training Accuracy :  0.9342139373296957
Test Accuracy :  0.6487491562090812
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.96      0.94   2930624
           1       0.96      0.91      0.93   2971310

    accuracy                           0.93   5901934
   macro avg       0.94      0.93      0.93   5901934
weighted avg       0.94      0.93      0.93   5901934



Precision (точность): Precision для класса 0 составляет 0.91, что означает, что 91% объектов, которые модель предсказала как класс 0, действительно принадлежат к классу 0. Precision для класса 1 составляет 0.96, что означает, что 96% объектов, которые модель предсказала как класс 1, действительно принадлежат к классу 1.
Recall (полнота): Recall для класса 0 составляет 0.96, что означает, что модель правильно идентифицировала 96% всех объектов класса 0. Recall для класса 1 составляет 0.91, что означает, что модель правильно идентифицировала 91% всех объектов класса 1.
F1-score: F1-score является гармоническим средним между precision и recall. F1-score для класса 0 равен 0.94, а для класса 1 - 0.93.
Support: Support представляет собой количество истинных ответов в каждом классе.
Accuracy (точность): Общая точность модели составляет 0.93, что означает, что модель правильно классифицировала 93% всех объектов.

Для начала посчитаю метрику по ответам, а затем по вероятностям ответов

In [ ]:
from sklearn.metrics import ndcg_score

relevance=y_test

ndcg_at_20 = ndcg_score([test_pred], [relevance], k=20)


print("NDCG@20:", ndcg_at_20)

NDCG@20: 0.6285848043844992


In [ ]:
from sklearn.metrics import ndcg_score

test_proba = DecisionTree.predict_proba(X_test)[:, 1]
relevance=y_test

ndcg_at_20_dt = ndcg_score([test_proba], [relevance], k=20)


print("NDCG@20:", ndcg_at_20_dt)

NDCG@20: 0.650873787694193


**lightgbm**

Для градиентного бустинга я выбрала библиотеку lightgbm, так как пользуесь ей наиболее часто. Подобрала гиперпараметры так, чтобы максимизировать метрику оценивания.

In [13]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

params = {
    'objective': 'binary', # бинарная классификация
    'metric': 'binary_error', # метрика - ошибка бинарной классификации
    'boosting_type': 'gbdt', # тип бустинга - градиентный бустинг деревьев решений
    'num_leaves': 31, # максимальное количество листьев в дереве
    'learning_rate': 0.001, # скорость обучения
    'feature_fraction': 0.9, # доля признаков, используемых для обучения каждого дерева
    'bagging_fraction': 0.8, # доля обучающих данных, используемых для обучения каждого дерева
    'bagging_freq': 5, # частота bagging
    'verbose': 0 # уровень вывода информации
}


lgb_model = lgb.train(params, train_data, valid_sets=[test_data])

In [ ]:
y_pred_lgb = lgb_model.predict(X_test)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred_lgb]

train_pred = lgb_model.predict(X_train)
train_pred_binary = [1 if pred > 0.5 else 0 for pred in train_pred]
train_acc = accuracy_score(y_train, train_pred_binary)
print("Training Accuracy:", train_acc)
test_acc_lgb = accuracy_score(y_test, y_pred_binary)
print("Test Accuracy:", test_acc_lgb)
print("Classification Report:\n", classification_report(y_test, y_pred_binary))

Training Accuracy: 0.6272789563556624
Test Accuracy: 0.6274883360307533
Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.58      0.61    732138
           1       0.62      0.67      0.65    743346

    accuracy                           0.63   1475484
   macro avg       0.63      0.63      0.63   1475484
weighted avg       0.63      0.63      0.63   1475484



In [ ]:

predicted_relevance = y_pred_lgb
relevance = y_test

ndcg_at_20_lgb = ndcg_score([predicted_relevance], [relevance], k=20)


print("NDCG@20:", ndcg_at_20_lgb)

NDCG@20: 0.9706004869519037


В случае с LightGBM, мы используйем прямо результаты predict для расчёта NDCG, поскольку это уже вероятности класса 1.


Для Decision Tree и других аналогичных моделей в scikit-learn, мы извлекаем вероятности класса 1 из результата predict_proba.


Это различие в подходах связано с разными реализациями методов предсказания в разных библиотеках.

In [ ]:
import numpy as np
unique_values, counts = np.unique(y_pred_binary, return_counts=True)

# Выводим уникальные значения и их количество
for value, count in zip(unique_values, counts):
    print(f"Значение {value}: {count} раз")

Значение 0: 665795 раз
Значение 1: 809689 раз


**Логрег**

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
y_pred = model.predict(X_test)
train_pred = model.predict(X_train)
train_acc = accuracy_score(y_train, train_pred)
print("Training Accuracy:", train_acc)
test_acc_logreg = accuracy_score(y_test, y_pred)
print("Test Accuracy:", test_acc)
print("Classification Report:\n", classification_report(y_test, y_pred))

Training Accuracy: 0.6254351539681738
Test Accuracy: 0.6261436925103898
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.64      0.63    732138
           1       0.63      0.61      0.62    743346

    accuracy                           0.63   1475484
   macro avg       0.63      0.63      0.63   1475484
weighted avg       0.63      0.63      0.63   1475484



In [ ]:
import numpy as np
unique_values, counts = np.unique(y_pred, return_counts=True)

# Выводим уникальные значения и их количество
for value, count in zip(unique_values, counts):
    print(f"Значение {value}: {count} раз")

Значение 0: 757565 раз
Значение 1: 717919 раз


In [ ]:
from sklearn.metrics import ndcg_score

predicted_relevance = model.predict_proba(X_test)[:, 1]

relevance=y_test

ndcg_at_20_logreg = ndcg_score([predicted_relevance], [relevance], k=20)


print("NDCG@20:", ndcg_at_20_logreg)

NDCG@20: 0.7097831465350183


# ОЦЕНКА КАЧЕСТВА

NDCG@20 (Normalized Discounted Cumulative Gain at 20) - это метрика, которая используется для оценки качества ранжирования результатов в поисковой или рекомендательной системе.

  Мною было принятно решение использовать вероятности вместо прямых предсказаний (бинаризованных значений). Это особенно важно при расчете метрик, оценивающих качество ранжирования, таких как NDCG (Normalized Discounted Cumulative Gain). Вот несколько ключевых причин, по которым стоит предпочесть вероятности:


*   Более точное ранжирование:
Вероятности предоставляют градацию уверенности модели в каждом из предсказаний. Это позволяет более точно ранжировать объекты, так как модель может быть на 80% уверена в принадлежности одного объекта к классу и на 60% — в другого. Эти различия в уверенности пропадают, если использовать только итоговые классы (0 или 1), где все положительные примеры считаются равноценными.
*   Улучшенное качество метрики:
NDCG рассчитывается на основе качества ранжирования, где релевантные (или более релевантные) объекты должны находиться выше в списке. Использование вероятностей позволяет этой метрике более точно отразить, насколько хорошо модель умеет различать между более и менее релевантными объектами.


*   Информативность предсказаний:
Вероятности предоставляют больше информации о предсказаниях, чем простые бинарные метки. Это особенно ценно в приложениях, где важно не только знать, к какому классу относится объект, но и насколько велика вероятность этого. Например, в медицинских диагностических системах или в финансовых приложениях, где степень риска или уверенности может критически влиять на принятие решений.

*   Флексибильность в интерпретации:
При работе с несбалансированными данными, где классы могут значительно отличаться по размеру, вероятности помогают сохранить чувствительность к меньшему классу, который может быть недопредставлен в бинарных предсказаниях.
Подходит для оптимизации порогов:
Работая с вероятностями, можно оптимизировать порог классификации для достижения наилучшего баланса между различными типами ошибок (например, между точностью и полнотой), что невозможно, если используются только финальные классы.











In [ ]:
metrics_df = pd.DataFrame({
    'Decision Tree': [DecisionTree_test_acc, ndcg_at_20_dt],
    'LGB': [test_acc_lgb, ndcg_at_20_lgb],
    'LogReg': [test_acc_logreg, ndcg_at_20_logreg]
}, index=['Test Accuracy', 'NDCG@20 Score'])

In [ ]:
metrics_df = metrics_df.T
sorted_df = metrics_df.sort_values('NDCG@20 Score', ascending=False)

In [ ]:
sorted_df

,Test Accuracy,NDCG@20 Score
LGB,0.627488,0.970600
LogReg,0.626144,0.709783
Decision Tree,0.648749,0.650874


Нетрудно заметить, что лучший скор получается у модели LGB. С помощью нее мы и сгенерируем ответы для тестового файла.

In [27]:
!7z x /content/drive/MyDrive/test.csv.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/                                 1 file, 43925208 bytes (42 MiB)

Extracting archive: /content/drive/MyDrive/test.csv.7z
--
Path = /content/drive/MyDrive/test.csv.7z
Type = 7z
Physical Size = 43925208
Headers Size = 122
Method = LZMA2:24
Solid = -
Blocks = 1

  0%      2% - test.csv                 4% - test.csv                 8% - test.csv                11% - test.csv                15% - test.csv                19% - test.csv                20% - test.csv    

In [14]:
test_data=pd.read_csv('test.csv')

In [ ]:
test_data.shape

(2556790, 6)

In [15]:
test_songs=pd.read_csv('songs.csv')

In [16]:
members_data=pd.read_csv('members.csv')

In [ ]:
test_songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [31]:
test_data.head()

,id,msno,song_id,source_system_tab,source_screen_name,source_type
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,song-based-playlist
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio


In [17]:
test_data = test_data.merge(test_songs, on='song_id', how='left')

test_data = test_data.merge(members_data, on='msno', how='left')

In [33]:
test_data.shape

(2556790, 18)

In [34]:
test_data.head()

,id,msno,song_id,source_system_tab,source_screen_name,source_type,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,registration_init_time,expiration_date
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library,224130.0,458,梁文音 (Rachel Liang),Qi Zheng Zhang,NaN,3.0,1,0,NaN,7,20160219,20170918
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library,320470.0,465,林俊傑 (JJ Lin),林俊傑,孫燕姿/易家揚,3.0,1,0,NaN,7,20160219,20170918
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,song-based-playlist,315899.0,2022,Yu Takahashi (高橋優),Yu Takahashi,Yu Takahashi,17.0,1,0,NaN,4,20161117,20161124
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio,285210.0,465,U2,The Edge| Adam Clayton| Larry Mullen| Jr.,NaN,52.0,3,30,male,9,20070725,20170430
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio,197590.0,873,Yoga Mr Sound,Neuromancer,NaN,-1.0,3,30,male,9,20070725,20170430


Далее проделаем с тестовыми данными всю ту же работу, что и проводили с обучающими данными в самом начале.

In [ ]:
test_data.isnull().any()

id                        False
msno                      False
song_id                   False
source_system_tab          True
source_screen_name         True
source_type                True
song_length                True
genre_ids                  True
artist_name                True
composer                   True
lyricist                   True
language                   True
city                      False
bd                        False
gender                     True
registered_via            False
registration_init_time    False
expiration_date           False
dtype: bool

In [18]:
test_data[['genre_ids','composer','lyricist']] = test_data[['genre_ids','composer','lyricist']].fillna('UnKnown')
test_data[['language','genre_ids']] = test_data[['language','genre_ids']].fillna(0)

test_data[['gender']] = test_data[['gender']].fillna(0)

test_data[['source_type', 'source_screen_name', 'source_system_tab']] = test_data[['source_type', 'source_screen_name', 'source_system_tab']].fillna('UnKnown')

In [19]:
mean_song_length = test_data['song_length'].mean()
test_data['song_length'] = test_data['song_length'].fillna(mean_song_length)
test_data['song_length'] = test_data['song_length'].astype(int)
mean_song_length = test_data['language'].mean()
test_data['language'] = test_data['language'].fillna(mean_song_length)
test_data['language'] = test_data['language'].astype(int)

In [20]:
test_data=test_data.drop(columns=['msno', 'song_id','artist_name','composer','lyricist'])

In [39]:
test_data=test_data.drop(columns=['source_screen_name_People global', 'source_screen_name_People local'])

In [35]:
test_data=test_data.drop(columns=['id'])

In [21]:
import pandas as pd

def convert_first_id(x):
    x = str(x)
    try:
        if '|' in x:
            return int(x.split('|')[0])
        return int(x)
    except ValueError:
        return 0


In [22]:
test_data['genre_ids'] = test_data['genre_ids'].apply(convert_first_id)

In [40]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556790 entries, 0 to 2556789
Data columns (total 54 columns):
 #   Column                                   Dtype
---  ------                                   -----
 0   song_length                              int64
 1   genre_ids                                int64
 2   language                                 int64
 3   city                                     int64
 4   bd                                       int64
 5   registered_via                           int64
 6   registration_init_time                   int64
 7   expiration_date                          int64
 8   source_system_tab_UnKnown                bool 
 9   source_system_tab_discover               bool 
 10  source_system_tab_explore                bool 
 11  source_system_tab_listen with            bool 
 12  source_system_tab_my library             bool 
 13  source_system_tab_notification           bool 
 14  source_system_tab_radio                  bool 
 15

In [37]:
test_data.isnull().any()

song_length                                False
genre_ids                                  False
language                                   False
city                                       False
bd                                         False
registered_via                             False
registration_init_time                     False
expiration_date                            False
source_system_tab_UnKnown                  False
source_system_tab_discover                 False
source_system_tab_explore                  False
source_system_tab_listen with              False
source_system_tab_my library               False
source_system_tab_notification             False
source_system_tab_radio                    False
source_system_tab_search                   False
source_system_tab_settings                 False
source_screen_name_Album more              False
source_screen_name_Artist more             False
source_screen_name_Concert                 False
source_screen_name_D

In [24]:
test_data = pd.get_dummies(test_data, columns=['source_system_tab', 'source_screen_name', 'source_type', 'gender'])

In [41]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556790 entries, 0 to 2556789
Data columns (total 54 columns):
 #   Column                                   Dtype
---  ------                                   -----
 0   song_length                              int64
 1   genre_ids                                int64
 2   language                                 int64
 3   city                                     int64
 4   bd                                       int64
 5   registered_via                           int64
 6   registration_init_time                   int64
 7   expiration_date                          int64
 8   source_system_tab_UnKnown                bool 
 9   source_system_tab_discover               bool 
 10  source_system_tab_explore                bool 
 11  source_system_tab_listen with            bool 
 12  source_system_tab_my library             bool 
 13  source_system_tab_notification           bool 
 14  source_system_tab_radio                  bool 
 15

In [34]:
test_data.shape

(2556790, 57)

In [42]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_test_data = scaler.transform(test_data)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [43]:
import pandas as pd

y_pred_lgb = lgb_model.predict(X_test_data)

indices = pd.Series(range(len(y_pred_lgb)), name='Index')

predictions_df = pd.DataFrame({'Index': indices, 'Prediction': y_pred_lgb})

predictions_df.to_csv('predictions.csv', index=False)

print("Predictions saved to predictions.csv")


Predictions saved to predictions.csv


In [50]:
predictions_df.rename(columns={'Index': 'id', 'Prediction': 'target'}, inplace=True)

predictions_df.to_csv('predictions.csv', index=False)

print("Predictions saved to predictions.csv")

Predictions saved to predictions.csv


In [51]:
predictions_df.head()

,id,target
0,0,0.519891
1,1,0.519891
2,2,0.497024
3,3,0.498984
4,4,0.498984


In [52]:
predictions_df.shape

(2556790, 2)

In [ ]:
!7z x /content/drive/MyDrive/sample_submission.csv.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/                                 1 file, 463688 bytes (453 KiB)

Extracting archive: /content/drive/MyDrive/sample_submission.csv.7z
--
Path = /content/drive/MyDrive/sample_submission.csv.7z
Type = 7z
Physical Size = 463688
Headers Size = 146
Method = LZMA2:24
Solid = -
Blocks = 1

  0%     99% - sample_submission.csv                            Everything is Ok

Size:       29570380
Compressed: 463688


In [46]:
sample_submission = pd.read_csv('sample_submission.csv')

In [47]:
sample_submission.head()

,id,target
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [48]:
sample_submission.shape

(2556790, 2)